In [38]:
import pandas as pd
import json
import os
import re

In [39]:
pickle_files = ["./data/Sonnet.pkl", "./data/GPT-4o.pkl", "./data/gemini-1.5-pro.pkl"]

dfs = [pd.read_pickle(file) for file in pickle_files]
df = pd.concat(dfs)

In [40]:
df["is_prediction_correct"] = df["gt"] == df["predicted"]

# Group by model and word, then calculate the accuracy
accuracy_per_model_and_word = df.groupby(["Model", "word"])[
    "is_prediction_correct"
].mean()

# Convert the accuracy to percentage and print the result
accuracy_per_model_and_word = accuracy_per_model_and_word * 100
# round 2 decimal places
accuracy_per_model_and_word = accuracy_per_model_and_word.round(2)

print(accuracy_per_model_and_word)

Model           word                
GPT-4o          Acknowledgement         68.63
                Subdermatoglyphic       63.77
                tHyUiKaRbNqWeOpXcZvM    78.53
Gemini-1.5-Pro  Acknowledgement         98.33
                Subdermatoglyphic       91.42
                tHyUiKaRbNqWeOpXcZvM    92.92
Sonnet          Acknowledgement         26.67
                Subdermatoglyphic       23.53
                tHyUiKaRbNqWeOpXcZvM    48.75
Name: is_prediction_correct, dtype: float64


In [44]:
print(accuracy_per_model_and_word.to_latex())

\begin{tabular}{llr}
\toprule
 &  & is_prediction_correct \\
Model & word &  \\
\midrule
\multirow[t]{3}{*}{GPT-4o} & Acknowledgement & 68.630000 \\
 & Subdermatoglyphic & 63.770000 \\
 & tHyUiKaRbNqWeOpXcZvM & 78.530000 \\
\cline{1-3}
\multirow[t]{3}{*}{Gemini-1.5-Pro} & Acknowledgement & 98.330000 \\
 & Subdermatoglyphic & 91.420000 \\
 & tHyUiKaRbNqWeOpXcZvM & 92.920000 \\
\cline{1-3}
\multirow[t]{3}{*}{Sonnet} & Acknowledgement & 26.670000 \\
 & Subdermatoglyphic & 23.530000 \\
 & tHyUiKaRbNqWeOpXcZvM & 48.750000 \\
\cline{1-3}
\bottomrule
\end{tabular}



In [36]:
# Assuming 'df' has a column named 'font' that contains the font information for each entry.

# Group by model, word, and font, then calculate the accuracy
accuracy_per_model_word_font = df.groupby(["Model", "word", "font_path"])[
    "is_prediction_correct"
].mean()

# Convert the accuracy to percentage and round to 2 decimal places
accuracy_per_model_word_font = (accuracy_per_model_word_font * 100).round(2)

print(accuracy_per_model_word_font)

Model           word                  font_path                                
GPT-4o          Acknowledgement       fonts/Helvetica.ttf                          69.66
                                      fonts/OpenSans-VariableFont_wdth,wght.ttf    67.60
                Subdermatoglyphic     fonts/Helvetica.ttf                          67.98
                                      fonts/OpenSans-VariableFont_wdth,wght.ttf    59.50
                tHyUiKaRbNqWeOpXcZvM  fonts/Helvetica.ttf                          82.28
                                      fonts/OpenSans-VariableFont_wdth,wght.ttf    74.79
Gemini-1.5-Pro  Acknowledgement       fonts/Helvetica.ttf                          98.89
                                      fonts/OpenSans-VariableFont_wdth,wght.ttf    97.78
                Subdermatoglyphic     fonts/Helvetica.ttf                          91.18
                                      fonts/OpenSans-VariableFont_wdth,wght.ttf    91.67
                tHyUiKaRbNqWeO

In [45]:
print(accuracy_per_model_word_font.to_latex())

\begin{tabular}{lllr}
\toprule
 &  &  & is_prediction_correct \\
Model & word & font_path &  \\
\midrule
\multirow[t]{6}{*}{GPT-4o} & \multirow[t]{2}{*}{Acknowledgement} & fonts/Helvetica.ttf & 69.660000 \\
 &  & fonts/OpenSans-VariableFont_wdth,wght.ttf & 67.600000 \\
\cline{2-4}
 & \multirow[t]{2}{*}{Subdermatoglyphic} & fonts/Helvetica.ttf & 67.980000 \\
 &  & fonts/OpenSans-VariableFont_wdth,wght.ttf & 59.500000 \\
\cline{2-4}
 & \multirow[t]{2}{*}{tHyUiKaRbNqWeOpXcZvM} & fonts/Helvetica.ttf & 82.280000 \\
 &  & fonts/OpenSans-VariableFont_wdth,wght.ttf & 74.790000 \\
\cline{1-4} \cline{2-4}
\multirow[t]{6}{*}{Gemini-1.5-Pro} & \multirow[t]{2}{*}{Acknowledgement} & fonts/Helvetica.ttf & 98.890000 \\
 &  & fonts/OpenSans-VariableFont_wdth,wght.ttf & 97.780000 \\
\cline{2-4}
 & \multirow[t]{2}{*}{Subdermatoglyphic} & fonts/Helvetica.ttf & 91.180000 \\
 &  & fonts/OpenSans-VariableFont_wdth,wght.ttf & 91.670000 \\
\cline{2-4}
 & \multirow[t]{2}{*}{tHyUiKaRbNqWeOpXcZvM} & fonts/Helveti